In [ ]:
from vnstock3 import Vnstock
stock = Vnstock().stock(symbol='VCI', source='VCI')

: 

In [63]:
symbols = stock.listing.all_symbols()
symbols = symbols.rename(columns={"ticker": "symbol"})


In [64]:
# symbols.to_csv("symbols_all.csv")

## Extract erroneous scraped symbols
We use this code to extract symbols that encountered errors during the scraping process. Subsequently, we will re-scrape these symbols

In [74]:
import re
import os
import pandas as pd
import numpy as np

def extract_symbols(log_folder_path):
    warning_symbols = set()
    info_symbols = set()

    # Regex patterns to identify WARNING and INFO logs
    warning_pattern = re.compile(r"WARNING - Not enough reports (\w+):")
    info_pattern = re.compile(r"INFO - Scraping Successfully (\w+):")

    # Loop through each log file in the folder
    for filename in os.listdir(log_folder_path):
        if filename.endswith(".log"):
            with open(os.path.join(log_folder_path, filename), 'r') as file:
                for line in file:
                    # Match WARNING logs
                    warning_match = warning_pattern.search(line)
                    if warning_match:
                        symbol = warning_match.group(1)
                        warning_symbols.add(symbol)

                    # Match INFO logs
                    info_match = info_pattern.search(line)
                    if info_match:
                        symbol = info_match.group(1)
                        info_symbols.add(symbol)

    # Exclude symbols that appear in INFO logs from WARNING symbols
    final_warning_symbols = warning_symbols - info_symbols
    print("Total Scraped symbols: ", len(list(final_warning_symbols)) + len(list(info_symbols)))
    info_symbols1 = info_symbols.copy()
    # Check folders in crawl_table for symbols in info_symbols
    for symbol in info_symbols:
        try:
            symbol_path = os.path.join('./crawl_table/', symbol)
            success_df = pd.read_csv(symbol_path + "/success.csv")
            if len(success_df) != len(success_df["STT"].unique()):
                final_warning_symbols.add(symbol)
                info_symbols1.remove(symbol)
                print("duplicate in success symbols: ")
                print(symbol)
        except:
            final_warning_symbols.add(symbol)
            info_symbols1.remove(symbol)
            print("Dont have success.csv: ")
            print(symbol)

    # Create a DataFrame to display the results
    warning_df = pd.DataFrame(final_warning_symbols, columns=["symbol"])
    info_df = pd.DataFrame(info_symbols1, columns=["symbol"])
    print("Symbols with warnings (excluding those in INFO logs):")
    print(warning_df)

    return warning_df, info_df



In [75]:
# Example usage
log_folder_path = "./logging"  # Replace with your log folder path
warnings_df, info_df = extract_symbols(log_folder_path)

Total Scraped symbols:  1594
Dont have success.csv: 
V15
Dont have success.csv: 
AGE
Dont have success.csv: 
HAC
Dont have success.csv: 
VKP
Dont have success.csv: 
TCI
Dont have success.csv: 
XDC
Dont have success.csv: 
GTT
Dont have success.csv: 
TVB
Dont have success.csv: 
APG
Dont have success.csv: 
WSS
Dont have success.csv: 
PSI
Dont have success.csv: 
VUA
Dont have success.csv: 
CTN
Dont have success.csv: 
DSC
Dont have success.csv: 
STL
Dont have success.csv: 
PX1
Dont have success.csv: 
B82
Dont have success.csv: 
S96
Dont have success.csv: 
SBS
Dont have success.csv: 
BVS
Dont have success.csv: 
MCD
Dont have success.csv: 
APS
Dont have success.csv: 
HBS
Dont have success.csv: 
VFS
duplicate in success symbols: 
ACS
duplicate in success symbols: 
LCC
duplicate in success symbols: 
NDP
Dont have success.csv: 
AGR
Dont have success.csv: 
KTW
Dont have success.csv: 
IVS
Dont have success.csv: 
PSG
Dont have success.csv: 
VDS
Dont have success.csv: 
AAS
Dont have success.csv: 
VN

In [76]:
len(list(set(symbols['symbol']) - set(list(info_df['symbol']))))

96

In [68]:
symbol_dir = os.path.join("E:/PUBLIC_DATA_PROJECT/Optimizing_scraping/", "crawl_table")


In [69]:
len([d for d in os.listdir(symbol_dir) if os.path.isdir(os.path.join(symbol_dir, d))])

1524

In [70]:
symbol_error = symbols.loc[~symbols["symbol"].isin(list(info_df['symbol']))].reset_index(drop=True).sort_values(by=["symbol"], ascending=False)
symbol_error

,symbol,organ_name
101,YEG,Công ty Cổ phần Tập đoàn Yeah1
100,XDC,Công ty Cổ phần Xây dựng Công trình Tân Cảng
99,WSS,Công ty Cổ phần Chứng khoán Phố Wall
98,VUA,Công ty Cổ phần Chứng khoán Stanley Brothers
97,VNZ,Công ty Cổ phần VNG
...,...,...
4,AGR,Công ty Cổ phần Chứng khoán AGRIBANK
3,AGE,Công ty Cổ phần Môi trường Đô thị An Giang
2,ACS,Công ty Cổ phần Xây lắp Thương mại 2
1,ABW,Công ty Cổ phần Chứng khoán An Bình


In [71]:
symbol_error.loc[symbol_error.symbol.str.contains("VSN")]

,symbol,organ_name


In [72]:
import json
import shutil

for symbol in list(symbol_error["symbol"]):
    try:
        checkpoint_path = f"./checkpoints/checkpoint_{symbol}.json"
        checkpoint = {"current_page": 1, "last_row_index": 0}
        crawl_table_path = f"./crawl_table/{symbol}"

        # Save the updated checkpoints back to the file
        with open(checkpoint_path, 'w') as f:
            json.dump(checkpoint, f)

        # Xóa thư mục symbol nếu tồn tại
        if os.path.exists(crawl_table_path) and os.path.isdir(crawl_table_path):
            shutil.rmtree(crawl_table_path)
            print(f"Đã xóa thư mục: {crawl_table_path}")
    except:
        continue

Đã xóa thư mục: ./crawl_table/VNZ
Đã xóa thư mục: ./crawl_table/VGI
Đã xóa thư mục: ./crawl_table/VEA
Đã xóa thư mục: ./crawl_table/TDW
Đã xóa thư mục: ./crawl_table/SJD
Đã xóa thư mục: ./crawl_table/SDC
Đã xóa thư mục: ./crawl_table/S55
Đã xóa thư mục: ./crawl_table/PVI
Đã xóa thư mục: ./crawl_table/PEG
Đã xóa thư mục: ./crawl_table/NVB
Đã xóa thư mục: ./crawl_table/NDP
Đã xóa thư mục: ./crawl_table/LIG
Đã xóa thư mục: ./crawl_table/LHG
Đã xóa thư mục: ./crawl_table/LCC
Đã xóa thư mục: ./crawl_table/ICG
Đã xóa thư mục: ./crawl_table/HHS
Đã xóa thư mục: ./crawl_table/HEV
Đã xóa thư mục: ./crawl_table/GPC
Đã xóa thư mục: ./crawl_table/GLW
Đã xóa thư mục: ./crawl_table/DTD
Đã xóa thư mục: ./crawl_table/D11
Đã xóa thư mục: ./crawl_table/CRE
Đã xóa thư mục: ./crawl_table/CRC
Đã xóa thư mục: ./crawl_table/CPI
Đã xóa thư mục: ./crawl_table/CLW
Đã xóa thư mục: ./crawl_table/CGV
Đã xóa thư mục: ./crawl_table/BTN
Đã xóa thư mục: ./crawl_table/ACS


In [ ]:
# symbol_error["symbol"].to_csv("error_symbols.csv")

In [37]:
len("Báo cáo tài chính Riêng - bán niên năm 2022_ kèm Gỉai trình lợi nhuận sau thuế TNDN tại Báo cáo KQHĐKD sau kiểm toán của kỳ công bố thay đổi từ 10% trở lên so với báo cáo cùng kỳ năm trước")

188

In [9]:
import pandas as pd
pd.read_csv("E:\PUBLIC_DATA_PROJECT\Optimizing_scraping\crawl_table\ACG\success.csv")

,Unnamed: 0,STT,Tên báo cáo,Đơn vị,Trích yếu,Thời gian gửi,Mã doanh nghiệp,Tên công ty,Tiêu đề,Saving_path
0,0,1,Báo cáo tài chính Mẹ - Quý,CTCP Gỗ An Cường,BCTC Công ty mẹ Quý 3 năm 2024,28/10/2024 00:00:00,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Mẹ - Quý,./crawl_table/ACG/BCTC Công ty mẹ Quý 3 năm 20...
1,1,2,Báo cáo tài chính Hợp nhất - Quý,CTCP Gỗ An Cường,BCTC hợp nhất Quý 3 năm 2024,28/10/2024 00:00:00,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Hợp nhất - Quý,./crawl_table/ACG/BCTC hợp nhất Quý 3 năm 2024...
2,2,3,Báo cáo tài chính Hợp nhất - Bán niên,CTCP Gỗ An Cường,Báo cáo tài chính soát xét hợp nhất bán niên 2024,29/08/2024 00:00:00,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Hợp nhất - Bán niên,./crawl_table/ACG/Báo cáo tài chính soát xét h...
3,3,4,Báo cáo tài chính Mẹ - Bán niên,CTCP Gỗ An Cường,Báo cáo tài chính soát xét Riêng bán niên 2024,29/08/2024 00:00:00,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Mẹ - Bán niên,./crawl_table/ACG/Báo cáo tài chính soát xét R...
4,4,5,Báo cáo tài chính Mẹ - Quý 2/ 2024,CTCP Gỗ An Cường,Báo cáo tài chính Riêng Quý 2 năm 2024,30/07/2024 13:04:36,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Mẹ - Quý 2/ 2024,./crawl_table/ACG/Báo cáo tài chính Riêng Quý ...
...,...,...,...,...,...,...,...,...,...,...
71,71,72,Báo cáo tài chính Hợp nhất quý I/ 2019,CTCP Gỗ An Cường,Báo cáo tài chính hợp nhất Quý 1 năm 2019 [Đã ...,20/02/2020 10:07:39,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Hợp nhất quý I/ 2019,./crawl_table/ACG/Báo cáo tài chính hợp nhất Q...
72,72,73,Báo cáo tài chính Riêng quý I/ 2019,CTCP Gỗ An Cường,Báo cáo tài chính riêng Quý 1 Năm 2019 [Đã côn...,20/02/2020 09:49:56,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Riêng quý I/ 2019,./crawl_table/ACG/Báo cáo tài chính riêng Quý ...
73,73,74,Báo cáo tài chính Riêng bán niên 2018,CTCP Gỗ An Cường,Báo cáo tài chính riêng bán niên năm 2018 (Côn...,20/02/2020 09:46:31,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Riêng bán niên 2018,./crawl_table/ACG/Báo cáo tài chính riêng bán ...
74,74,75,Báo cáo tài chính Hợp nhất năm 2018,CTCP Gỗ An Cường,Báo cáo tài chính hợp nhất năm 2018 (sau kiểm ...,19/02/2020 14:48:17,3700748131,CTCP Gỗ An Cường,Báo cáo tài chính Hợp nhất năm 2018,./crawl_table/ACG/Báo cáo tài chính hợp nhất n...


In [81]:
import os
import pandas as pd

def collect_stock_ids(base_dir):
    data = []

    # Duyệt qua tất cả các thư mục con
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        
        if os.path.isdir(folder_path):
            success_path = os.path.join(folder_path, 'success.csv')
            warning_path = os.path.join(folder_path, 'warning.csv')

            # Đọc file và xử lý giá trị
            success_df = None
            warning_df = None

            if os.path.exists(success_path):
                success_values = pd.read_csv(success_path, dtype=object)['Mã doanh nghiệp'].astype(str)
                success_df = success_values.mode().iloc[0] if not success_values.empty else None
            
            if os.path.exists(warning_path):
                warning_values = pd.read_csv(warning_path, dtype=object)['Mã doanh nghiệp'].astype(str)
                warning_df = warning_values.mode().iloc[0] if not warning_values.empty else None

            # Gộp kết quả vào danh sách
            data.append({
                'Symbol': folder,
                'tax_code': success_df,
            })

    # Chuyển dữ liệu thành DataFrame
    df = pd.DataFrame(data)
    return df

# Đường dẫn tới thư mục crawl_table
base_dir = './crawl_table'
result_df = collect_stock_ids(base_dir)





In [82]:
result_df

,Symbol,tax_code
0,A32,0300517896
1,AAA,0800373586
2,AAM,1800448811
3,AAT,2800222245
4,AAV,0800819038
...,...,...
1520,XMP,04112020092714
1521,XPH,0100100311
1522,YBC,5200216647
1523,YBM,5200824701


In [83]:
result_df.loc[result_df["Symbol"] == "VIC"]

,Symbol,tax_code
1414,VIC,0101245486


In [ ]:
# result_df.to_csv("mapping_taxcode_symbol.csv")